In [6]:
import torch 
import geometric_kernels.torch 
from geometric_kernels.spaces import Hypersphere
from mdgp.models.deep_gps import GeometricManifoldDeepGP
from mdgp.utils import sphere_uniform_grid
from mdgp.samplers import sample_naive

In [2]:
# Single-output models with 0-3 hidden layers
x = sphere_uniform_grid(400)
inducing_points = sphere_uniform_grid(60)
space = Hypersphere(2)
models = [
    GeometricManifoldDeepGP(space=space, num_hidden=h, inducing_points=inducing_points)
    for h in range(8)
]

### It seems that every additional layer above 1 adds aroudn 0.51 seconds to the forward pass in inference mode

In [4]:
torch.set_grad_enabled(False)
for h, model in enumerate(models): 
    print(f"Hidden layers: {h}")
    # time model forward pass via timeit with 10 loops each 
    %timeit -n 10 model(x)

Hidden layers: 0
14.7 ms ± 4.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 1
149 ms ± 6.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 2
384 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 3
475 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 4
664 ms ± 37.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 5
871 ms ± 44.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 6
1.05 s ± 45.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 7
1.26 s ± 52.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### In training mode each additional layer adds around 

In [3]:
torch.set_grad_enabled(True)
for h, model in enumerate(models): 
    if h > 3: 
        break 
    print(f"Hidden layers: {h}")
    # time model forward pass via timeit with 10 loops each 
    %timeit -n 10 model(x)

Hidden layers: 0
11.1 ms ± 2.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 1
172 ms ± 5.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 2
532 ms ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 3
919 ms ± 73.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
torch.set_grad_enabled(True)
for h, model in enumerate(models): 
    if h > 4: 
        break 
    print(f"Hidden layers: {h}")
    # time model forward pass via timeit with 10 loops each 
    %timeit -n 10 sample_naive(model(x)).mean().backward()

Hidden layers: 0
30.2 ms ± 2.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 1
415 ms ± 8.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 2
1.33 s ± 55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 3
2.33 s ± 71.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hidden layers: 4
3.52 s ± 217 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Using vectorisation we speed up computation by around 5 times 